<a href="https://colab.research.google.com/github/simranroy01/lawBOT/blob/main/lawbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
import nltk
import string
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pickle

In [6]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\drbin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [7]:
# Load the legal dataset from CSV
df = pd.read_csv('ipc_sections.csv')

# Preprocess the dataset
df['text'] = df['Description'] + ' ' + df['Offense'] + ' ' + df['Punishment'] + ' ' + df['Section']
df['text'] = df['text'].astype(str)  # Convert all values to string

In [8]:
# Initialize NLTK resources
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

lemmer = nltk.stem.WordNetLemmatizer()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\drbin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\drbin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\drbin\AppData\Roaming\nltk_data...


In [9]:
# Function to lemmatize tokens
def LemToken(tokens):
    return [lemmer.lemmatize(token) for token in tokens]

remove_punc_dict = dict((ord(punct), None) for punct in string.punctuation)

In [10]:
# Function to lemmatize and normalize text
def LemNormalize(text):
    return LemToken(nltk.word_tokenize(text.lower().translate(remove_punc_dict)))

# Initialize TF-IDF Vectorizer
TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english', token_pattern=r'(?u)\b\w\w+\b')

# Fit TF-IDF Vectorizer on legal dataset text
tfidf_matrix = TfidfVec.fit_transform(df['text'])

c:\Python312\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\Python312\Lib\site-packages\sklearn\feature_extraction\text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  warnings.warn(


In [11]:
# Function to generate response using TF-IDF and cosine similarity
def response(user_response):
    user_response = ' '.join(nltk.word_tokenize(user_response.lower()))
    query_vector = TfidfVec.transform([user_response])
    cosine_similarities = cosine_similarity(query_vector, tfidf_matrix)
    idx = cosine_similarities.argsort()[0][-1]
    return df.iloc[idx]['text']

In [12]:
# Function to handle greetings
def greet(sentence):
    for word in sentence.split():
        if word.lower() in ('hello', 'hi', 'wassup', 'hey'):
            return random.choice(['hi', 'hey!', 'hey there!', 'hola user'])

In [13]:
# Main execution loop
flag = True
print('Bot: Hello, I am your legal chatbot. How can I help you?')
while flag:
    user_response = input('You: ')
    user_response = user_response.lower()
    if user_response != 'bye':
        if user_response == 'thank you' or user_response == 'thanks':
            flag = False
            print('Bot: You are welcome')
        else:
            if greet(user_response) is not None:
                print('Bot:', greet(user_response))
            else:
                bot_response = response(user_response)
                print('Bot:', bot_response)
    else:
        flag = False
        print('Bot: Goodbye!!')

Bot: Hello, I am your legal chatbot. How can I help you?
Bot: hola user


KeyboardInterrupt: Interrupted by user